In [1]:
import keras

In [2]:
from keras.datasets import mnist
from keras.models import Model
import keras.layers as L
from keras.callbacks import RemoteMonitor

In [40]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [41]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [42]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(10)

In [43]:
y_train.shape = (-1, 1)
y_test.shape = (-1, 1)

In [44]:
y_train = enc.fit_transform(y_train)
y_test = enc.transform(y_test)
y_train.shape, y_test.shape

((60000, 10), (10000, 10))

In [7]:
def make_model():
    X_inp = L.Input([28, 28])
    X = L.Flatten()(X_inp)
    X = L.Dense(512, activation='relu')(X)
    X = L.Dense(256, activation='relu')(X)
    X = L.Dense(64, activation='relu')(X)
    X = L.Dense(10, activation='sigmoid')(X)
    model = Model(inputs=X_inp, outputs=X)
    return model

In [8]:
model = make_model()
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_3 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_4 (Dense)              (None, 10)                650       
Total params: 550,346
Trainable params: 550,346
Non-trainable params: 0
_____________________________________________________

In [46]:
model.compile('adam', loss='categorical_crossentropy', metrics=['acc'])

In [47]:
model.fit(x_train, y_train, validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 21s 353us/step - loss: 8.9134 - acc: 0.0988 - val_loss: 9.0495 - val_acc: 0.0980


In [58]:
monitor = RemoteMonitor(root="http://localhost:5000", path="/", 
                        headers={"Content-type":"application/json"} ,send_as_json=True)

In [61]:
model.fit(x_test, y_test, callbacks=[monitor])

Epoch 1/1
10000/10000 [==============================] - 3s 310us/step - loss: 9.0495 - acc: 0.0980


In [87]:
import requests, json

In [111]:
class ModelTracker(keras.callbacks.Callback):
    def __init__(self, model_key):
        self.__url = "http://localhost:5000/"
        self.__json_header = {'Content-type':'application/json'}
        self.model_key = model_key
        req = requests.post(self.__url + "auth", json={'model_key':model_key}, 
                      headers=self.__json_header)
        if req.status_code != 200:
            raise Exception("Invalid model key")
            return
        super().__init__()
        
    def on_train_begin(self, logs={}):
        req = requests.post(self.__url + "start", json=self.params, 
                            headers=self.__json_header)
        self.training_id = req.json()['training_id'] 
    
    def on_train_end(self, logs={}):
        requests.post(self.__url + "end", json={'training_id' : self.training_id},
                     headers=self.__json_header)
        
    def on_epoch_begin(self, epoch, logs={}):
        logs['training_id'] = self.training_id
        logs['epoch'] = epoch
        requests.post(self.__url + "epochbegin", data=json.dumps(str(logs)), headers=self.__json_header)
    
    def on_epoch_end(self, epoch, logs={}):
        logs['training_id'] = self.training_id
        logs['epoch'] = epoch
        requests.post(self.__url + "epochend", data=json.dumps(str(logs)), headers=self.__json_header)
        

In [112]:
track = ModelTracker("varun")

In [113]:
model.fit(x_test, y_test, validation_data=(x_test, y_test), callbacks=[track], epochs=3)

Train on 10000 samples, validate on 10000 samples
Epoch 1/3
10000/10000 [==============================] - 4s 411us/step - loss: 9.0495 - acc: 0.0980 - val_loss: 9.0495 - val_acc: 0.0980
Epoch 2/3
10000/10000 [==============================] - 4s 403us/step - loss: 9.0495 - acc: 0.0980 - val_loss: 9.0495 - val_acc: 0.0980
Epoch 3/3
10000/10000 [==============================] - 4s 408us/step - loss: 9.0495 - acc: 0.0980 - val_loss: 9.0495 - val_acc: 0.0980


In [114]:
track.params

{'batch_size': 32,
 'epochs': 3,
 'steps': None,
 'samples': 10000,
 'verbose': 1,
 'do_validation': True,
 'metrics': ['loss', 'acc', 'val_loss', 'val_acc']}